In [1]:
import xarray as xr
import geopandas as gpd
import glob

In [2]:
ug = gpd.read_file(
    "C:/Users/Administrator/OneDrive/Desktop/rainfall_prediction/uganda_administrative_boundaries/simplified_uganda_districts.shp"
)

ug.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [3]:
ug.head()

,ADM2_EN,ADM1_EN,geometry
0,Abim,Northern,"POLYGON ((33.58099 3.14912, 33.58555 3.14702, ..."
1,Adjumani,Northern,"POLYGON ((32.04597 3.58896, 32.04578 3.58875, ..."
2,Agago,Northern,"POLYGON ((33.41652 3.30434, 33.41701 3.30403, ..."
3,Alebtong,Northern,"POLYGON ((33.03629 2.5013, 33.03664 2.50075, 3..."
4,Amolatar,Northern,"POLYGON ((32.9014 1.8057, 32.90231 1.80568, 32..."


In [4]:
ds = xr.open_dataset(
    "C:/Users/Administrator/OneDrive/Desktop/rainfall_prediction/era5/wind_north_south/wind_northsouth_direction2020_2025.grib",
    engine="cfgrib",
)

ds

<xarray.Dataset> Size: 1GB
Dimensions:     (time: 51249, latitude: 24, longitude: 259)
Coordinates:
  * time        (time) datetime64[ns] 410kB 2020-01-01 ... 2025-11-05T08:00:00
  * latitude    (latitude) float64 192B 4.25 4.0 3.75 3.5 ... -1.0 -1.25 -1.5
  * longitude   (longitude) float64 2kB -29.5 -29.25 -29.0 ... 34.5 34.75 35.0
    number      int64 8B ...
    step        timedelta64[ns] 8B ...
    surface     float64 8B ...
    valid_time  (time) datetime64[ns] 410kB ...
Data variables:
    v10         (time, latitude, longitude) float32 1GB ...
Attributes:
    GRIB_edition:            1
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2025-11-20T14:14 GRIB to CDM+CF via cfgrib-0.9.1...

In [5]:
minx, miny, maxx, maxy = ug.total_bounds

ds_ug = ds.sel(longitude=slice(minx, maxx), latitude=slice(maxy, miny))
ds_ug

<xarray.Dataset> Size: 100MB
Dimensions:     (time: 51249, latitude: 22, longitude: 22)
Coordinates:
  * time        (time) datetime64[ns] 410kB 2020-01-01 ... 2025-11-05T08:00:00
  * latitude    (latitude) float64 176B 4.0 3.75 3.5 3.25 ... -0.75 -1.0 -1.25
  * longitude   (longitude) float64 176B 29.75 30.0 30.25 ... 34.5 34.75 35.0
    number      int64 8B ...
    step        timedelta64[ns] 8B ...
    surface     float64 8B ...
    valid_time  (time) datetime64[ns] 410kB ...
Data variables:
    v10         (time, latitude, longitude) float32 99MB ...
Attributes:
    GRIB_edition:            1
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2025-11-20T14:14 GRIB to CDM+CF via cfgrib-0.9.1...

In [6]:
ds_monthly = ds_ug.resample(time="1M").mean()
ds_monthly

c:\Program Files\Python311\Lib\site-packages\xarray\groupers.py:530: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  self.index_grouper = pd.Grouper(


<xarray.Dataset> Size: 138kB
Dimensions:    (time: 71, latitude: 22, longitude: 22)
Coordinates:
  * time       (time) datetime64[ns] 568B 2020-01-31 2020-02-29 ... 2025-11-30
  * latitude   (latitude) float64 176B 4.0 3.75 3.5 3.25 ... -0.75 -1.0 -1.25
  * longitude  (longitude) float64 176B 29.75 30.0 30.25 ... 34.5 34.75 35.0
    number     int64 8B 0
    step       timedelta64[ns] 8B 00:00:00
    surface    float64 8B 0.0
Data variables:
    v10        (time, latitude, longitude) float32 137kB -1.556 ... -0.3478
Attributes:
    GRIB_edition:            1
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2025-11-20T14:14 GRIB to CDM+CF via cfgrib-0.9.1...

In [7]:
files = glob.glob(
    "C:/Users/Administrator/OneDrive/Desktop/rainfall_prediction/era5/wind_north_south/*.grib"
)

datasets = []

for file in files:
    ds = xr.open_dataset(file, engine="cfgrib")
    minx, miny, maxx, maxy = ug.total_bounds
    ds_ug = ds.sel(longitude=slice(minx, maxx), latitude=slice(maxy, miny))
    ds_monthly = ds_ug.resample(time="1M").mean()
    datasets.append(ds_monthly)

combined_ds = xr.concat(datasets, dim="time")

combined_ds

c:\Program Files\Python311\Lib\site-packages\xarray\groupers.py:530: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  self.index_grouper = pd.Grouper(
c:\Program Files\Python311\Lib\site-packages\xarray\groupers.py:530: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  self.index_grouper = pd.Grouper(
c:\Program Files\Python311\Lib\site-packages\xarray\groupers.py:530: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  self.index_grouper = pd.Grouper(
Ignoring index file 'C:/Users/Administrator/OneDrive/Desktop/rainfall_prediction/era5/wind_north_south\\wind_northsouth_direction2020_2025.grib.5b7b6.idx' incompatible with GRIB file
c:\Program Files\Python311\Lib\site-packages\xarray\groupers.py:530: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  self.index_grouper = pd.Grouper(


<xarray.Dataset> Size: 1MB
Dimensions:    (time: 539, latitude: 22, longitude: 22)
Coordinates:
  * time       (time) datetime64[ns] 4kB 1981-01-31 1981-02-28 ... 2025-11-30
  * latitude   (latitude) float64 176B 4.0 3.75 3.5 3.25 ... -0.75 -1.0 -1.25
  * longitude  (longitude) float64 176B 29.75 30.0 30.25 ... 34.5 34.75 35.0
    number     int64 8B 0
    step       timedelta64[ns] 8B 00:00:00
    surface    float64 8B 0.0
Data variables:
    v10        (time, latitude, longitude) float32 1MB -0.04278 ... -0.3478
Attributes:
    GRIB_edition:            1
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2025-11-20T14:17 GRIB to CDM+CF via cfgrib-0.9.1...

In [8]:
df_wind_v_component = combined_ds.to_dataframe().reset_index()
df_wind_v_component

,time,latitude,longitude,v10,number,step,surface
0,1981-01-31,4.00,29.75,-0.042781,0,0 days,0.0
1,1981-01-31,4.00,30.00,-0.233930,0,0 days,0.0
2,1981-01-31,4.00,30.25,-0.403055,0,0 days,0.0
3,1981-01-31,4.00,30.50,-0.604413,0,0 days,0.0
4,1981-01-31,4.00,30.75,-0.722725,0,0 days,0.0
...,...,...,...,...,...,...,...
260871,2025-11-30,-1.25,34.00,-0.619629,0,0 days,0.0
260872,2025-11-30,-1.25,34.25,-0.448303,0,0 days,0.0
260873,2025-11-30,-1.25,34.50,-0.426549,0,0 days,0.0
260874,2025-11-30,-1.25,34.75,-0.408692,0,0 days,0.0


In [9]:
# convert grid cells points to shapely points
gdf = gpd.GeoDataFrame(
    df_wind_v_component,
    geometry=gpd.points_from_xy(
        df_wind_v_component.longitude, df_wind_v_component.latitude
    ),
    crs="EPSG:4326",
)

In [10]:
gdf = gpd.sjoin(gdf, ug, how="left", predicate="within")
gdf

,time,latitude,longitude,v10,number,step,surface,geometry,index_right,ADM2_EN,ADM1_EN
0,1981-01-31,4.00,29.75,-0.042781,0,0 days,0.0,POINT (29.75 4),NaN,NaN,NaN
1,1981-01-31,4.00,30.00,-0.233930,0,0 days,0.0,POINT (30 4),NaN,NaN,NaN
2,1981-01-31,4.00,30.25,-0.403055,0,0 days,0.0,POINT (30.25 4),NaN,NaN,NaN
3,1981-01-31,4.00,30.50,-0.604413,0,0 days,0.0,POINT (30.5 4),NaN,NaN,NaN
4,1981-01-31,4.00,30.75,-0.722725,0,0 days,0.0,POINT (30.75 4),NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
260871,2025-11-30,-1.25,34.00,-0.619629,0,0 days,0.0,POINT (34 -1.25),NaN,NaN,NaN
260872,2025-11-30,-1.25,34.25,-0.448303,0,0 days,0.0,POINT (34.25 -1.25),NaN,NaN,NaN
260873,2025-11-30,-1.25,34.50,-0.426549,0,0 days,0.0,POINT (34.5 -1.25),NaN,NaN,NaN
260874,2025-11-30,-1.25,34.75,-0.408692,0,0 days,0.0,POINT (34.75 -1.25),NaN,NaN,NaN


In [11]:
df_monthly = gdf.groupby(["ADM2_EN", "ADM1_EN", "time"])["v10"].mean().reset_index()
df_monthly

,ADM2_EN,ADM1_EN,time,v10
0,Abim,Northern,1981-01-31,-0.469906
1,Abim,Northern,1981-02-28,-0.506172
2,Abim,Northern,1981-03-31,-0.028231
3,Abim,Northern,1981-04-30,0.326704
4,Abim,Northern,1981-05-31,0.582360
...,...,...,...,...
64675,Zombo,Northern,2025-07-31,0.505518
64676,Zombo,Northern,2025-08-31,0.172524
64677,Zombo,Northern,2025-09-30,-0.059908
64678,Zombo,Northern,2025-10-31,0.092298


In [12]:
# rename columns
df_monthly = df_monthly.rename(
    columns={
        "ADM2_EN": "District",
        "ADM1_EN": "Region",
        "time": "Date",
        "v10": "Wind_V_Component",
    }
)
df_monthly

,District,Region,Date,Wind_V_Component
0,Abim,Northern,1981-01-31,-0.469906
1,Abim,Northern,1981-02-28,-0.506172
2,Abim,Northern,1981-03-31,-0.028231
3,Abim,Northern,1981-04-30,0.326704
4,Abim,Northern,1981-05-31,0.582360
...,...,...,...,...
64675,Zombo,Northern,2025-07-31,0.505518
64676,Zombo,Northern,2025-08-31,0.172524
64677,Zombo,Northern,2025-09-30,-0.059908
64678,Zombo,Northern,2025-10-31,0.092298


In [13]:
# save to csv
df_monthly.to_csv("uganda_monthly_wind_v_component_1981_2025.csv", index=False)